In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
cols = pd.read_csv("oct_all_20200225_masked.csv").columns.tolist()
rnflcols = {}
for col in cols[6:7]+cols[8:11]+cols[13:22]+ cols[29:-1]:
    rnflcols[col] = 'float'
for col in cols[:1]+cols[4:6]+cols[12:13]+cols[7:8]+cols[22:29]+cols[-1:]:
    rnflcols[col] = 'str'
df = pd.read_csv("oct_all_20200225_masked.csv",dtype=rnflcols, parse_dates=cols[1:4])

In [3]:
ids = ["VIP008044","VIP061418","VIP037173","VIP047216","VIP031886",
"VIP056062","VIP033021","VIP014389","VIP016537","VIP018334","VIP041440",
"VIP037986","VIP041609","VIP057901","VIP058564","VIP066946","VIP067499",
"VIP069109" ,"VIP071560" ,"VIP090530","VIP091026","VIP097007","VIP111183",
"VIP114093","VIP120493","VIP022706","VIP099137","VIP110041","VIP070097",
"VIP037986" ,"VIP093776","VIP007879","VIP019124","VIP020446","VIP019218",
"VIP037748","VIP053755","VIP081666","VIP089742","VIP114093","VIP119064",
"VIP122233","VIP053142","VIP009947","VIP026016","VIP043105","VIP043105",
"VIP118474","VIP014250" ,"VIP010831","VIP014198","VIP002538","VIP010378"]

rol = ["L","L","L","L","L","R","R","R","R","R","R","R","R","R","R","R","R",
"R","R","R","R","R","R","R","R","R","R","R","L","L","L","L","L","L","L","L",
"L","L","L","L","L","L"]

idrnfl = [98,92,67,49,116]

criteria1, criteria2, criteria3 = [], [], []
for val1, val2, val3 in zip(ids[:len(idrnfl)], rol[:len(idrnfl)], idrnfl):
    criteria1 += [val1+val2+str(float(val3))]

for val1, val2 in zip(ids[len(idrnfl):len(rol)], rol[len(idrnfl):]):
    criteria2 += [val1+val2]

criteria3 = ids[len(rol):]

In [4]:
df['age'] = (pd.to_datetime(df['examdate_shift']) - pd.to_datetime(df['dob_deduce_shift'])).dt.days / 365.25
df['crt1'] = df['maskedid']+df['eye']+df['rnflmean_g'].astype('str')
df['crt2'] = df['maskedid']+df['eye']
df['crt3'] = df['maskedid']

In [5]:
df = df[(df['scantype'] != "OCT Radial Circle Scan") & (df['diametermm'] < 4) & (df['age'] > 18) & 
            (df['age'] < 100) & (df['quality'] >15) & (df['rnflmean_g'] > 20) & (df['rnflmean_g'] < 150) &
            (~df['crt1'].isin(criteria1)) & (~df['crt2'].isin(criteria2)) & (~df['crt3'].isin(criteria3))]


df = df.drop(columns=['age','crt1','crt2','crt3'])

df = df.loc[(df[cols[29:-1]] != 0).all(axis=1)]
df = df.drop(columns=['dob_shift', 'mergedby', 'imageid', 'scantype', 'examtime', 'aqmversion', 'rnflclass_g', 'rnflclass_t', 'rnflclass_ts', 'rnflclass_ti', 'rnflclass_n', 'rnflclass_ns', 'rnflclass_ni','satellite'])

In [6]:
cols1 = df.columns.to_list()
agg_func = {}
for col in cols1[:2]+cols1[3:4]:
    agg_func[col] = 'first'
for col in cols1[4:]:
    agg_func[col] = 'mean'

In [7]:
grouped = df.groupby('eye')
grouped = iter(grouped)
_, df_Leye = next(grouped)
_, df_Reye = next(grouped)
df_Reye = df_Reye.reset_index().drop(columns=['index'])
df_Leye = df_Leye.reset_index().drop(columns=['index'])

In [8]:
grouped = df_Reye.groupby('maskedid')

singlevals = []

appenddata = []
for name, tempdf in grouped:
    if tempdf.shape[0] == 1:
        tempdf['days'] = (tempdf['examdate_shift'] - tempdf['examdate_shift'].iloc[0]).dt.days
        tempdf['progress'] = 999.0
        tempdf['label'] = 'b'
        singlevals+= [name]
    else:
        tempdf = tempdf.groupby(tempdf[cols1[2]]).aggregate(agg_func).reset_index().reindex(columns=tempdf.columns)
        if tempdf.shape[0] == 1:
            tempdf['days'] = (tempdf['examdate_shift'] - tempdf['examdate_shift'].iloc[0]).dt.days
            tempdf['progress'] = 999.0
            tempdf['label'] = 'b'
            singlevals+= [name]
        else:
            tempdf['days'] = (tempdf['examdate_shift'] - tempdf['examdate_shift'].iloc[0]).dt.days
            tempdf['progress'] = 999.0
            tempdf['progress'].iloc[1:] = tempdf['rnflmean_g'].iloc[1:] - tempdf['rnflmean_g'].shift().iloc[1:]
            tempdf['label'] = tempdf["progress"].apply(lambda x: 'b' if x == 999.0  else ('p' if x < 0 else 'n'))
    
    appenddata += [tempdf]
n_dfR = pd.concat(appenddata)
    

In [9]:
grouped = df_Leye.groupby('maskedid')

appenddata = []
for name, tempdf in grouped:
    if tempdf.shape[0] == 1:
        tempdf['days'] = (tempdf['examdate_shift'] - tempdf['examdate_shift'].iloc[0]).dt.days
        tempdf['progress'] = 999.0
        tempdf['label'] = 'b'
        singlevals+= [name]
    else:
        tempdf = tempdf.groupby(tempdf[cols1[2]]).aggregate(agg_func).reset_index().reindex(columns=tempdf.columns)
        if tempdf.shape[0] == 1:
            tempdf['days'] = (tempdf['examdate_shift'] - tempdf['examdate_shift'].iloc[0]).dt.days
            tempdf['progress'] = 999.0
            tempdf['label'] = 'b'
            singlevals+= [name]
        else:
            tempdf['days'] = (tempdf['examdate_shift'] - tempdf['examdate_shift'].iloc[0]).dt.days
            tempdf['progress'] = 999.0
            tempdf['progress'].iloc[1:] = tempdf['rnflmean_g'].iloc[1:] - tempdf['rnflmean_g'].shift().iloc[1:]
            tempdf['label'] = tempdf["progress"].apply(lambda x: 'b' if x == 999.0  else ('p' if x < 0 else 'n'))
    
    appenddata += [tempdf]
n_dfL = pd.concat(appenddata)

In [10]:
df_comb = pd.concat([n_dfR,n_dfL])
df_comb['age'] = (df_comb['examdate_shift'] - df_comb['dob_deduce_shift']).dt.days/365.25
df_comb = df_comb.reindex(columns=cols1[:4]+['days','age','progress']+cols1[4:]+['label'])


In [11]:
df_comb.to_csv('octdataset.csv')

In [16]:
df_comb['progress'][df_comb['progress']==999.0] = 0.0

In [18]:
new_df = df_comb.reset_index().drop(columns=['index'])
new_df.to_csv('octdataset.csv')